In [1]:

import random
from ..ToolParametriser import *

from datetime import datetime

from absl import app
from absl import flags


SyntaxError: invalid syntax (<ipython-input-1-8edf1fee0853>, line 2)

In [4]:
class BenchmarkingToolCreator(ABC):
    print("in benchmarking tool creator")
    """
    The BenchmarkingToolCreator class declares the factory method that is supposed to return an
    object of a Job class. The BenchmarkingToolCreator's subclasses usually provide the
    implementation of this method.
    """

    def __init__(
        self,
        BenchmarkingCSVFile_path,
        InputFiles_path,
        storage_path,
        Number_of_jobs_repetition,
    ):
        self.BenchmarkingCSVFile_path = BenchmarkingCSVFile_path
        self.InputFiles_path = InputFiles_path
        self.storage_path = storage_path
        self.Number_of_jobs_repetition = Number_of_jobs_repetition

    @abstractmethod
    def factory_method_create_job(self):
        """
        Note that the BenchmarkingToolCreator may also provide some default implementation of
        the factory method.
        """
        pass

    def runBenchmarking(self) -> str:
        print("benchmarkingtoolcreator runBenchmarking")
        
        """
        Also note that, despite its name, the BenchmarkingToolCreator's primary responsibility
        is not creating jobs. Usually, it contains some core business logic
        that relies on Job objects, returned by the factory method.
        Subclasses can indirectly change that business logic by overriding the
        factory method and returning a different type of job from it.
        """

        # Call the factory method to create a Job object.
        job = self.factory_method_create_job()
        # Now, use the job.

        # TODO: Inside of this method will be our core business logic
        
        # current date and time
        now = datetime.now()

        # ID to identify each Benchmarking executed
        ExecutionID = now.strftime("%Y%m%d%H%M%S")

        # Storing job_parameters of CSV file
        job_parameters = job.readBenchmarkingProfiles()

        # Specific files Identification (.tsv, .d, .xml, .fasta)
        specificInputFiles = job.identifySpecificInputFiles()

        # Let's run the job according to the number of repetition
        for parameters in job_parameters:

            for _ in range(0, self.Number_of_jobs_repetition):
                now = datetime.now()  # current date and time
                JobExecutionID = now.strftime("%Y%m%d%H%M%S")
                running_job_path = f"{self.storage_path}repo-{parameters['job-name']}-{JobExecutionID}/"

                job.createRepository(running_job_path)

                # Copy samples files k = number of input files to randomly select
                sample_files = random.sample(specificInputFiles["original_files"], k=int(parameters["NumFiles"]))
                specificInputFiles["sample_files"] = sample_files

                for sample_file_path in specificInputFiles["sample_files"]:
                    name_of_folder = sample_file_path.split("/")[-1]
                    job.copyInputFiles(sample_file_path, running_job_path + name_of_folder)

                # Copy ONLY specific input files such (.tsv, .xml, .fasta)
                job.copySpecificInputFiles(specificInputFiles, running_job_path)
        
                # Create and Execute the SBatch File

                job.createExecutableBatchFile(
                    parameters,
                    running_job_path,
                    specificInputFiles,
                    ExecutionID,
                )

        
        # result = f"BenchmarkingToolCreator: The same creator's code has just worked with {job.readBenchmarkingProfiles()}"
        result = "BenchMe has finished running"

        return result


in benchmarking tool creator


In [8]:
"""
Concrete Creators override the factory method in order to change the resulting
product's type.
"""

class MQBenchmarkingTool(BenchmarkingToolCreator):
    """
    Note that the signature of the method still uses the abstract job type,
    even though the concrete job is actually returned from the method. This
    way the BenchmarkingToolCreator can stay independent of concrete job classes.
    """
    def __init__(self,BenchmarkingCSVFile_path, InputFiles_path, storage_path, Number_of_jobs_repetition,):
        super().__init__(BenchmarkingCSVFile_path, InputFiles_path, storage_path, Number_of_jobs_repetition)

        # TODO: Variables needs to be initialize
        # Extract the list of Input filenames: .Fasta, .XML and .d
        # original_files = glob.glob(InputFiles_path + "*.d", recursive=False)

        # Create a Dictionary to store Input Files Orderly
        # MaxQuantInputFiles = {}
        # MaxQuantInputFiles["fasta_file"] = glob.glob(InputFiles_path + "*.fasta", recursive=False)[0]
        # MaxQuantInputFiles["xml_file"] = glob.glob(InputFiles_path + "*.xml", recursive=False)[0]

        # self.sample_files = sample_files
        # self.xml_file_path = xml_file_path
        # self.numthreads = numthreads
        
    def factory_method_create_job(self) -> Job:
        return MaxQuantJob(self.BenchmarkingCSVFile_path,self.InputFiles_path,self.storage_path,self.Number_of_jobs_repetition )


class DiaNNBenchmarkingTool(BenchmarkingToolCreator):
    def __init__(self,BenchmarkingCSVFile_path, InputFiles_path, storage_path, Number_of_jobs_repetition):
        super().__init__(BenchmarkingCSVFile_path, InputFiles_path, storage_path, Number_of_jobs_repetition)
        print("diannbenchmarkingtook def init")
        
    def factory_method_create_job(self) -> Job:
        print("diannbenchmarkingtook def factory method create job")
        
        return DiaNNJob(self.BenchmarkingCSVFile_path,self.InputFiles_path,self.storage_path,self.Number_of_jobs_repetition)
    
class GenericBenchmarkingTool(BenchmarkingToolCreator):
    print("In generic benchmarking tool with the mactory method create job")
    def __init__(self,BenchmarkingCSVFile_path, InputFiles_path, storage_path, Number_of_jobs_repetition):
        super().__init__(BenchmarkingCSVFile_path, InputFiles_path, storage_path, Number_of_jobs_repetition)
        
    def factory_method_create_job(self) -> Job:
        script_dir = input("Please enter the full directory of the location where the script is - ")
        params = input("Please enter any extra parameters to add to the scipt when run - ")
        return GenericJob(self.BenchmarkingCSVFile_path,self.InputFiles_path,self.storage_path,self.Number_of_jobs_repetition, script_dir, params)


In generic benchmarking tool with the mactory method create job


In [6]:
# TODO: Add this function to your diagram
def client_code(creator: BenchmarkingToolCreator) -> None:
    print("client code creator")
    """
    The client code works with an instance of a concrete creator, albeit through
    its base interface. As long as the client keeps working with the creator via
    the base interface, you can pass it any creator's subclass.
    """

    print(
            
            f"Client: I'm not aware of the creator's class, but it still works.\n"
            f"{creator.runBenchmarking()}",
            end="",
        )


In [7]:
# Welcoming message
BenchTool = input("Thanks for using BenchMe! Please select '1' if you want benchmark MQ, '2' for DiaNN, or '3' if you want a generic benchmark.")

InputFiles_path = input("Absolute path of input files (.d, .fasta, .tsv). I.E: '/stornext/HPCScratch/home/romano.h/Software-Projects/Local-Repositories/DiaNN/DiaNNFiles-Dataset/'")
BenchmarkingCSVFile_path = input("Name of CSV file with profiles for benchmarking. I.E: 'benchmarking-profiles.csv'")
storage_path = input("Absolute path of storage directory to save outputs. I.E: '/vast/scratch/users/romano.h/DiaNNBenchmarking/'")
Number_of_jobs_repetition = int(input("Number of times to run each benchmarking profile job. Default: 5 times"))

if BenchTool == '1':
    print("App: Launched with the MQBenchmarkingTool.")
    client_code(MQBenchmarkingTool(BenchmarkingCSVFile_path, InputFiles_path, storage_path, Number_of_jobs_repetition))
elif BenchTool == '2':
    print("App: Launched with the DiaNNBenchmarkingTool.")
    client_code(DiaNNBenchmarkingTool(BenchmarkingCSVFile_path, InputFiles_path, storage_path, Number_of_jobs_repetition))
elif BenchTool == '3':
    print("App: Launched with the GenericBenchmarkingTool.")
    client_code(GenericBenchmarkingTool(BenchmarkingCSVFile_path, InputFiles_path, storage_path, Number_of_jobs_repetition))
else:
    print("I am sorry, the selected option is invalid.")


KeyboardInterrupt: Interrupted by user

## This will need to be added/modified when migrating out of Notebooks

In [ ]:
# TODO: Add this function to your diagram
def main() -> None:
    """
    The client code works with an instance of a concrete creator, albeit through
    its base interface. As long as the client keeps working with the creator via
    the base interface, you can pass it any creator's subclass.
    """

    print("App: Launched with the MQBenchmarkingTool.")
    client_code(MQBenchmarkingTool())
    print("\n")

    print("App: Launched with the DiaNNBenchmarkingTool.")
    client_code(DiaNNBenchmarkingTool())

    def client_code(creator: BenchmarkingToolCreator):

        print(
            f"Client: I'm not aware of the creator's class, but it still works.\n"
            f"{creator.runBenchmarking()}",
            end="",
        )

In [ ]:
FLAGS = flags.FLAGS
flags.DEFINE_string("files", None, "Absolute path of Input DiaNN input files (.d, .fasta, .tsv). I.E: '/stornext/HPCScratch/home/romano.h/Software-Projects/Local-Repositories/DiaNN/DiaNNFiles-Dataset/'")
flags.DEFINE_string("profiles", None, "Name of CSV file with DiaNN Profiles for benchmarking. I.E: 'benchmarking-profiles.csv'")
flags.DEFINE_string("storage", None, "Absolute path of storage directory to save outputs. I.E: '/vast/scratch/users/romano.h/DiaNNBenchmarking/'")
flags.DEFINE_integer("repeat", 5,"Number of times to run each benchmarking profile job. Default: 5 times")

# Required flag.
flags.mark_flag_as_required("files")
flags.mark_flag_as_required("profiles")
flags.mark_flag_as_required("storage")

In [ ]:
if __name__ == "__main__":
    app.run(main)
